# Left / Right hand detection

Sometimes the detection is wrong
It mixes up left / right hand

In [24]:
import cv2
import mediapipe as mp
import numpy as np

from util.utils import detectHands

In [3]:
# Order: Blue, Green, Red
DOT_COLOR = (0, 0, 255)
CONNECTION_COLOR = (0, 255, 0)

TEXT_COLOR = (255, 255, 255)

EXIT_KEY = 'q'

In [25]:
def getLabel(loopIndex, hand, results, width, height):
    WRIST = 0
    output = None

    for classification in results.multi_handedness:
        clf = classification.classification[0]
        if (clf.index == loopIndex):
            label = clf.label
            score = clf.score

            text = f'{label}: {score:.2f}'

            lm = hand.landmark
            coords = np.multiply(
                np.array((lm[WRIST].x, lm[WRIST].y)), [width, height]
            ).astype(int)

            coords = tuple(coords)

            output = [text, coords]
            break
    
    return output

In [26]:
mpDrawing = mp.solutions.drawing_utils
mpHands = mp.solutions.hands

cap = cv2.VideoCapture(0)

with mpHands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:

    while (cap.isOpened()):
        success, frame = cap.read()

        # Camera's width & height
        width  = int(cap.get(3))
        height = int(cap.get(4))

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)
        
        image, results = detectHands(hands, frame)

        landmarks = results.multi_hand_landmarks
        if (landmarks):
            for index, hand in enumerate(landmarks):
                # Draws the detection on the image
                mpDrawing.draw_landmarks(
                    image, hand, mpHands.HAND_CONNECTIONS,
                    mpDrawing.DrawingSpec(color=DOT_COLOR, thickness=2, circle_radius=4),
                    mpDrawing.DrawingSpec(color=CONNECTION_COLOR, thickness=2, circle_radius=2)
                )

                label = getLabel(index, hand, results, width, height)
                if (label):
                    text = label[0]
                    coords = label[1]

                    cv2.putText(image, text, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2, cv2.LINE_AA)
                    

        cv2.imshow('Left / Right hand detection', image)

        # Exit if user pressed 'q'
        if (cv2.waitKey(10) & 0xFF == ord(EXIT_KEY)):
            break

cap.release()
cv2.destroyAllWindows()

In [20]:
# How many hands were found in the last frame
len(results.multi_hand_landmarks)

2

In [23]:
print(results.multi_handedness[0])
print(results.multi_handedness[1])

classification {
  index: 0
  score: 0.9854701161384583
  label: "Left"
}

classification {
  index: 1
  score: 0.9234537482261658
  label: "Right"
}



In [28]:
print(results.multi_handedness[0].classification[0])

print(results.multi_handedness[0].classification[0].index)

index: 0
score: 0.9854701161384583
label: "Left"

0
